In [1]:
import torchvision
import torch
from scipy.optimize import linear_sum_assignment as Egervary
import os
import shapely
from shapely import Point
from shapely import Polygon
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_poly_coords(label_path, pred_path):
    labels = []
    preds = []
    label_list = sorted(os.listdir(label_path))
    pred_list = sorted(os.listdir(pred_path))

    label_names = [item[:6] for item in label_list]
    pred_names = [item[:6] for item in pred_list]

    poplist = []
    for i in range(len(label_names)):
        if label_names[i] not in pred_names:
            poplist.append(label_list[i])
    for item in poplist:
        label_list.pop(label_list.index(item))

    for name in label_list:
        with open(label_path+"/"+name) as file:
            items = []
            while True:
                line = file.readline().strip().split()
                if line == []:
                    break
                line = coords_to_poly(line[:8])
                items.append(line)
            labels.append(items)

    for name in pred_list:
        with open(pred_path+"/"+name) as file:
            items = []
            while True:
                line = file.readline().strip().split()
                if line == []:
                    break
                line = coords_to_poly(line[1:])
                items.append(line)
            preds.append(items)

    return labels, preds


def coords_to_poly(coords):
    point1 = Point(coords[0], coords[1])
    point2 = Point(coords[2], coords[3])
    point3 = Point(coords[4], coords[5])
    point4 = Point(coords[6], coords[7])
    return Polygon((point1, point2, point3, point4))

def IoU(A, B, epsilon = 1e-6):
    inter_area = A.intersection(B).area
    union_area = A.area+B.area-inter_area
    return inter_area/(union_area+epsilon)

def IoU_matrix(labels, preds):
    return np.array([[IoU(A, B) for A in preds] for B in labels])

def polygon_to_nparray(polygon):
    poly_coords = list(polygon.exterior.coords)
    coord_array = []
    for item in poly_coords:
        coord_array.append(item[0])
        coord_array.append(item[1])
    return coord_array

def a_b_angle(a, b, c):
    alpha_rad = np.arccos((a**2+b**2-c**2)/(2*a*b))
    return alpha_rad*180/np.pi

def distance_2d(x1, y1, x2, y2):
    return np.sqrt((x1-x2)**2+(y1-y2)**2)

def poly_shift_2d(object, shift_x, shift_y):
    for i in range(len(object)):
        if i%2==0: 
            object[i] += shift_x
        else:
            object[i] += shift_y
    return object


def theta_difference(A, B):
    A_coords = polygon_to_nparray(A)
    B_coords = polygon_to_nparray(B)

    center_A_x = (A_coords[0]+A_coords[4])/2
    center_A_y = (A_coords[1]+A_coords[5])/2
    center_B_x = (B_coords[0]+B_coords[4])/2
    center_B_y = (B_coords[1]+B_coords[5])/2

    A_coords = poly_shift_2d(A_coords, -center_A_x, -center_A_y)
    B_coords = poly_shift_2d(B_coords, -center_B_x, -center_B_y)

    point1_x = (A_coords[2]-A_coords[4])/2
    point1_y = (A_coords[3]-A_coords[5])/2
    point2_x = (B_coords[2]-B_coords[4])/2
    point2_y = (B_coords[3]-B_coords[5])/2

    a = distance_2d(point1_x, point1_y, 0, 0)
    b = distance_2d(point2_x, point2_y, 0, 0)
    c = distance_2d(point1_x, point1_y, point2_x, point2_y)
    angle = a_b_angle(a, b, c)

    return min(abs(angle), abs(90-angle), abs(180-angle), abs(270-angle))

def theta_diff_dataset(labels, predictions):
    diffs = []
    for i in range(len(predictions)):
        matrix = IoU_matrix(labels[i], predictions[i])
        indexes = Egervary(matrix, maximize = True)
        for j in range(len(indexes[0])):
            k = indexes[0][j]
            l = indexes[1][j]
            diff = theta_difference(labels[i][k], predictions[i][l])
            #if diff>20:
            #    print(f"Itt van baj {i}")
            diffs.append(diff)
    return diffs


In [ ]:
label_path = "label_path"
pred_path = "pred_path"

label, pred = get_poly_coords(label_path, pred_path)
diffsn = theta_diff_dataset(label, pred)